## Data Integration (Recreational Zones in Berlin):



### Playgrounds dataset: 
#### Source Link: https://fbinter.stadt-berlin.de/fb/index.jsp?loginkey=alphaDataStart&alphaDataId=s_spielplatzbestand@senstadt

In [9]:
import pandas as pd
from geopy.geocoders import Nominatim
import time

from sqlalchemy import create_engine



In [10]:
## Importing dataset from local source (computer)

df=pd.read_csv('/Users/msubhanzk/Downloads/2. Data Analysis Course/14. June 2025/1. Internship Documents/4. Week # 4/Editing Data/playground.csv')

## pd.set_option("display.max_rows", None)

df.head()   ## Showing the first five rows of the dataset.


,Technischer Schlüssel,Schlüssel,Objektnummer,Bezirk,Ortsteil,Art der Grünanlage,Name des Spielplatzes,Namenszusatz des Spielplatzes,Baujahr,letztes Sanierungsjahr,Größe in m² (Kataster),Widmung,Nummer des Planungsraumes,Name des Planungsraumes,Nettospielfläche in m²
0,00008100_002f2c6b,00008100:002f2c6b,19441,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,-,-,-,-,-,-,-
1,00008100_0033cfac,00008100:0033cfac,35098,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,-,-,5031.90,-,12200413,Tegel Süd,-
2,00008100_0032a99a,00008100:0032a99a,21AS10,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,-,-,713.00,gewidmet,11300725,Nibelungenviertel,-
3,00008100_00199411,00008100:00199411,00156,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988,-,3934.00,gewidmet,10100104,Golliner Straße,-
4,00008100_00281623,00008100:00281623,150072,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,-,-,900.00,gewidmet,09301227,Karolinenhof,-


In [11]:
# Look at the columns'name. 

df.columns

Index(['Technischer Schlüssel', 'Schlüssel', 'Objektnummer', 'Bezirk',
       'Ortsteil', 'Art der Grünanlage', 'Name des Spielplatzes',
       'Namenszusatz des Spielplatzes', 'Baujahr', 'letztes Sanierungsjahr',
       'Größe in m² (Kataster)', 'Widmung', 'Nummer des Planungsraumes',
       'Name des Planungsraumes', 'Nettospielfläche in m²'],
      dtype='object')

### Convert the Column's name from German to English:


In [12]:

df.rename(columns={
    'Technischer Schlüssel': 'technical_id',
    'Schlüssel' : 'key',
    'Objektnummer':'object_number',
    'Bezirk':'district',
    'Ortsteil': 'neighborhood', 
    'Art der Grünanlage': 'green_area_type', 
    'Name des Spielplatzes': 'playground_name',
    'Namenszusatz des Spielplatzes': 'playground_suffix',
    'Baujahr': 'year_built',
    'letztes Sanierungsjahr': 'last_renovation_year',
    'Größe in m² (Kataster)': 'area_sqm',
    'Widmung':'dedication',
    'Nummer des Planungsraumes': 'planning_area_number',
    'Name des Planungsraumes': 'planning_area_name', 
    'Nettospielfläche in m²': 'net_play_area_sqm',
    
}, inplace = True)

In [13]:
# To see changes in the dataset columns'name.

df.columns

Index(['technical_id', 'key', 'object_number', 'district', 'neighborhood',
       'green_area_type', 'playground_name', 'playground_suffix', 'year_built',
       'last_renovation_year', 'area_sqm', 'dedication',
       'planning_area_number', 'planning_area_name', 'net_play_area_sqm'],
      dtype='object')

In [14]:
df.head()  ## To see the changes in the dataset.

,technical_id,key,object_number,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm
0,00008100_002f2c6b,00008100:002f2c6b,19441,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,-,-,-,-,-,-,-
1,00008100_0033cfac,00008100:0033cfac,35098,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,-,-,5031.90,-,12200413,Tegel Süd,-
2,00008100_0032a99a,00008100:0032a99a,21AS10,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,-,-,713.00,gewidmet,11300725,Nibelungenviertel,-
3,00008100_00199411,00008100:00199411,00156,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988,-,3934.00,gewidmet,10100104,Golliner Straße,-
4,00008100_00281623,00008100:00281623,150072,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,-,-,900.00,gewidmet,09301227,Karolinenhof,-


In [15]:
custom_ids = {
    'Mitte': 1,
    'Friedrichshain-Kreuzberg': 2,
    'Pankow': 3,
    'Charlottenburg-Wilmersdorf': 4,
    'Spandau': 5,
    'Steglitz-Zehlendorf': 6,
    'Tempelhof-Schöneberg': 7,
    'Neukölln': 8,
    'Treptow-Köpenick': 9,
    'Marzahn-Hellersdorf': 10,
    'Lichtenberg': 11,
    'Reinickendorf': 12
}
   
# Add a new column 'district_id' using the mapping

df['district_id'] = df['district'].map(custom_ids)

# Optional: view the mapping
print(custom_ids)

# Optional: view the result
df.head()

{'Mitte': 1, 'Friedrichshain-Kreuzberg': 2, 'Pankow': 3, 'Charlottenburg-Wilmersdorf': 4, 'Spandau': 5, 'Steglitz-Zehlendorf': 6, 'Tempelhof-Schöneberg': 7, 'Neukölln': 8, 'Treptow-Köpenick': 9, 'Marzahn-Hellersdorf': 10, 'Lichtenberg': 11, 'Reinickendorf': 12}


,technical_id,key,object_number,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,district_id
0,00008100_002f2c6b,00008100:002f2c6b,19441,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,-,-,-,-,-,-,-,7
1,00008100_0033cfac,00008100:0033cfac,35098,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,-,-,5031.90,-,12200413,Tegel Süd,-,12
2,00008100_0032a99a,00008100:0032a99a,21AS10,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,-,-,713.00,gewidmet,11300725,Nibelungenviertel,-,11
3,00008100_00199411,00008100:00199411,00156,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988,-,3934.00,gewidmet,10100104,Golliner Straße,-,10
4,00008100_00281623,00008100:00281623,150072,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,-,-,900.00,gewidmet,09301227,Karolinenhof,-,9


In [16]:
# Assume df already has 'district_id' column
cols = list(df.columns)          # Step 1
cols.remove('district_id')       # Step 2
cols.insert(3, 'district_id')    # Step 3: move to index 1 (second position)
df = df[cols]                    # Step 4
df.head()

,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,-,-,-,-,-,-,-
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,-,-,5031.90,-,12200413,Tegel Süd,-
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,-,-,713.00,gewidmet,11300725,Nibelungenviertel,-
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988,-,3934.00,gewidmet,10100104,Golliner Straße,-
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,-,-,900.00,gewidmet,09301227,Karolinenhof,-


In [17]:
unique_districts = df['district'].unique()
print(unique_districts)

['Tempelhof-Schöneberg' 'Reinickendorf' 'Lichtenberg'
 'Marzahn-Hellersdorf' 'Treptow-Köpenick' 'Spandau'
 'Friedrichshain-Kreuzberg' 'Mitte' 'Pankow' 'Neukölln'
 'Steglitz-Zehlendorf' 'Charlottenburg-Wilmersdorf']


In [18]:
## Creating a new column named 'full_address' 

df['full_address']=(df['playground_name'].astype(str)+ "," + df['planning_area_name'].astype(str)+ ', Berlin, Germany')

df.head()

,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,full_address
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,-,-,-,-,-,-,-,"Nuthestr. 26,-, Berlin, Germany"
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,-,-,5031.90,-,12200413,Tegel Süd,-,"Borsigdamm 4 (Spielplatz),Tegel Süd, Berlin, G..."
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,-,-,713.00,gewidmet,11300725,Nibelungenviertel,-,"Gudrunstr. 14 Spielplatz,Nibelungenviertel, Be..."
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988,-,3934.00,gewidmet,10100104,Golliner Straße,-,"Hohenwalder/Schorfheider Straße -Dreieck,Golli..."
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,-,-,900.00,gewidmet,09301227,Karolinenhof,-,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KS...


In [19]:
## To see the duplicates in the new created column. 

duplicates = df['full_address'].duplicated().sum()

print (f"There are {duplicates} duplicated address") 

There are 27 duplicated address


In [20]:
## drop duplicated addresses.

unique_addresses=df[['full_address']].drop_duplicates().copy()

df.head()

,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,full_address
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,-,-,-,-,-,-,-,"Nuthestr. 26,-, Berlin, Germany"
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,-,-,5031.90,-,12200413,Tegel Süd,-,"Borsigdamm 4 (Spielplatz),Tegel Süd, Berlin, G..."
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,-,-,713.00,gewidmet,11300725,Nibelungenviertel,-,"Gudrunstr. 14 Spielplatz,Nibelungenviertel, Be..."
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988,-,3934.00,gewidmet,10100104,Golliner Straße,-,"Hohenwalder/Schorfheider Straße -Dreieck,Golli..."
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,-,-,900.00,gewidmet,09301227,Karolinenhof,-,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KS...


### Finding the Latitude and Longtitude: 

In [21]:

geolocator = Nominatim(user_agent="berlin_playgrounds")
def geocode_playground(row):
    query = f"{row['playground_name']},{row['neighborhood']}, {row['district']}, Berlin, Germany"
    try:
        location = geolocator.geocode(query)
        time.sleep(1)  
        if location:
            return pd.Series([location.latitude, location.longitude])
    except:
        return pd.Series([None, None])

df[['latitude', 'longitude']] = df.apply(geocode_playground, axis=1)

df.head()

,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,full_address,latitude,longitude
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,-,-,-,-,-,-,-,"Nuthestr. 26,-, Berlin, Germany",52.393914,13.394292
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,-,-,5031.90,-,12200413,Tegel Süd,-,"Borsigdamm 4 (Spielplatz),Tegel Süd, Berlin, G...",NaN,NaN
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,-,-,713.00,gewidmet,11300725,Nibelungenviertel,-,"Gudrunstr. 14 Spielplatz,Nibelungenviertel, Be...",NaN,NaN
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988,-,3934.00,gewidmet,10100104,Golliner Straße,-,"Hohenwalder/Schorfheider Straße -Dreieck,Golli...",NaN,NaN
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,-,-,900.00,gewidmet,09301227,Karolinenhof,-,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KS...,NaN,NaN


## Cleaning the data:

In [22]:
## Here I did the following cleaning:

## Converts number-like columns to float

## Trims all text fields

# Identify numeric columns (based on cleaned names)

numeric_columns = ['year_built', 'last_renovation_year', 'area_sqm', 'net_play_area_sqm','planning_area_number']

# Convert these columns to numbers (if they exist)
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Strip whitespace from string fields
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df.head()


/var/folders/p3/p2wyc7s13wj812ps7spvh1c80000gn/T/ipykernel_40709/2021888107.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,full_address,latitude,longitude
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,-,NaN,NaN,NaN,-,NaN,-,NaN,"Nuthestr. 26,-, Berlin, Germany",52.393914,13.394292
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),-,NaN,NaN,5031.9,-,12200413.0,Tegel Süd,NaN,"Borsigdamm 4 (Spielplatz),Tegel Süd, Berlin, G...",NaN,NaN
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,-,NaN,NaN,713.0,gewidmet,11300725.0,Nibelungenviertel,NaN,"Gudrunstr. 14 Spielplatz,Nibelungenviertel, Be...",NaN,NaN
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988.0,NaN,3934.0,gewidmet,10100104.0,Golliner Straße,NaN,"Hohenwalder/Schorfheider Straße -Dreieck,Golli...",NaN,NaN
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,-,NaN,NaN,900.0,gewidmet,9301227.0,Karolinenhof,NaN,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KS...,NaN,NaN


In [23]:
## I did the following cleaning:
### Converting (-) to Unknown in two other columns.

df["playground_suffix"]=df["playground_suffix"].replace("-", "Unknown")
df["dedication"]=df["dedication"].replace("-", "Unknown")
df["planning_area_name"]=df["planning_area_name"].replace("-", "Unknown")
pd.set_option("display.max_rows", None)
df.head()

,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,full_address,latitude,longitude
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,Unknown,NaN,NaN,NaN,Unknown,NaN,Unknown,NaN,"Nuthestr. 26,-, Berlin, Germany",52.393914,13.394292
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),Unknown,NaN,NaN,5031.9,Unknown,12200413.0,Tegel Süd,NaN,"Borsigdamm 4 (Spielplatz),Tegel Süd, Berlin, G...",NaN,NaN
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,Unknown,NaN,NaN,713.0,gewidmet,11300725.0,Nibelungenviertel,NaN,"Gudrunstr. 14 Spielplatz,Nibelungenviertel, Be...",NaN,NaN
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988.0,NaN,3934.0,gewidmet,10100104.0,Golliner Straße,NaN,"Hohenwalder/Schorfheider Straße -Dreieck,Golli...",NaN,NaN
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,Unknown,NaN,NaN,900.0,gewidmet,9301227.0,Karolinenhof,NaN,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KS...,NaN,NaN


In [181]:
df.shape

(1879, 18)

In [24]:
df=df.drop_duplicates()
df.head()

,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,full_address,latitude,longitude
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,Unknown,NaN,NaN,NaN,Unknown,NaN,Unknown,NaN,"Nuthestr. 26,-, Berlin, Germany",52.393914,13.394292
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),Unknown,NaN,NaN,5031.9,Unknown,12200413.0,Tegel Süd,NaN,"Borsigdamm 4 (Spielplatz),Tegel Süd, Berlin, G...",NaN,NaN
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,Unknown,NaN,NaN,713.0,gewidmet,11300725.0,Nibelungenviertel,NaN,"Gudrunstr. 14 Spielplatz,Nibelungenviertel, Be...",NaN,NaN
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988.0,NaN,3934.0,gewidmet,10100104.0,Golliner Straße,NaN,"Hohenwalder/Schorfheider Straße -Dreieck,Golli...",NaN,NaN
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,Unknown,NaN,NaN,900.0,gewidmet,9301227.0,Karolinenhof,NaN,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KS...,NaN,NaN


In [27]:
print(df.dtypes)

technical_id             object
key                      object
object_number            object
district_id               int64
district                 object
neighborhood             object
green_area_type          object
playground_name          object
playground_suffix        object
year_built              float64
last_renovation_year    float64
area_sqm                float64
dedication               object
planning_area_number    float64
planning_area_name       object
net_play_area_sqm       float64
full_address             object
latitude                float64
longitude               float64
dtype: object


In [28]:
df.head()

,technical_id,key,object_number,district_id,district,neighborhood,green_area_type,playground_name,playground_suffix,year_built,last_renovation_year,area_sqm,dedication,planning_area_number,planning_area_name,net_play_area_sqm,full_address,latitude,longitude
0,00008100_002f2c6b,00008100:002f2c6b,19441,7,Tempelhof-Schöneberg,Lichtenrade,Spielplatz,Nuthestr. 26,Unknown,NaN,NaN,NaN,Unknown,NaN,Unknown,NaN,"Nuthestr. 26,-, Berlin, Germany",52.393914,13.394292
1,00008100_0033cfac,00008100:0033cfac,35098,12,Reinickendorf,Tegel,Spielplatz,Borsigdamm 4 (Spielplatz),Unknown,NaN,NaN,5031.9,Unknown,12200413.0,Tegel Süd,NaN,"Borsigdamm 4 (Spielplatz),Tegel Süd, Berlin, G...",NaN,NaN
2,00008100_0032a99a,00008100:0032a99a,21AS10,11,Lichtenberg,Lichtenberg,Spielplatz,Gudrunstr. 14 Spielplatz,Unknown,NaN,NaN,713.0,gewidmet,11300725.0,Nibelungenviertel,NaN,"Gudrunstr. 14 Spielplatz,Nibelungenviertel, Be...",NaN,NaN
3,00008100_00199411,00008100:00199411,00156,10,Marzahn-Hellersdorf,Marzahn,Spielplatz,Hohenwalder/Schorfheider Straße -Dreieck,seit 1990 pädagog. betreuter Spielplatz,1988.0,NaN,3934.0,gewidmet,10100104.0,Golliner Straße,NaN,"Hohenwalder/Schorfheider Straße -Dreieck,Golli...",NaN,NaN
4,00008100_00281623,00008100:00281623,150072,9,Treptow-Köpenick,Schmöckwitz,Spielplatz,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KSP,Unknown,NaN,NaN,900.0,gewidmet,9301227.0,Karolinenhof,NaN,Biebersdorfer Weg 12/ Lübbenauer Weg 23 Sch KS...,NaN,NaN


In [29]:
df.to_csv("playgrounds_Cleaned.csv", index = False)

